In [ ]:
!pip install dash==1.10.0

In [ ]:
!pip install dash dash-renderer dash-html-components dash-core-components plotly

In [ ]:
!pip install plotly

In [ ]:
!pip install dash-bootstrap-components

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import os
import base64
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import time

In [2]:

###################################
# Private function
###################################


def load_data(file_name, column_name):
    data = (
        pd.read_csv(file_name,)
        .melt(
            id_vars=["Province/State", "Country/Region", "Lat", "Long"],
            var_name="date",
            value_name=column_name,
        )
        .astype({"date": "str", column_name: int}, errors="ignore")
    )
    data["Province/State"].fillna("<all>", inplace=True)
    data[column_name].fillna(0, inplace=True)
    return data
# def loadData(fileName, columnName):
#     data = pd.read_csv(fileName) \
#              .drop(['Lat', 'Long'], axis=1) \
#              .melt(id_vars=['Province/State', 'Country/Region'], 
#                  var_name='date', value_name=columnName) \
#              .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
#                  errors='ignore')
#     data['Province/State'].fillna('<all>', inplace=True)
#     data[columnName].fillna(0, inplace=True)
#     return data


In [3]:
from datetime import datetime 

################################################################################
# Data processing
################################################################################

# Data and files working with
confirmed_cases = "time_series_covid19_confirmed_global.csv"
confirmed_deaths = "time_series_covid19_deaths_global.csv"
confirmed_recovered = "time_series_covid19_recovered_global.csv"

# Merge data frames into one
all_data = (
    load_data(confirmed_cases, "confirmed")
    .merge(load_data(confirmed_deaths, "deaths"))
    .merge(load_data(confirmed_recovered, "recovered"))
)

countries = all_data["Country/Region"].unique()
countries.sort()
all_data['date'] = pd.to_datetime(all_data['date'],errors='coerce')

date_options = [date_obj.strftime('%Y-%m-%d') for date_obj in all_data["date"].dt.date.unique()]
#date_options.sort()
date_options.sort(key = lambda date: datetime.strptime(date, '%Y-%m-%d')) 

all_data['date'] = all_data['date'].dt.strftime('%Y-%m-%d')
#print(all_data['date'])
# print(all_data)

# # Save numbers into variables
# latest_date = all_data["date"].max()

# # number of cases
# confirmed_cases_sum = sum_data(all_data, latest_date, 0, "confirmed")
# cases_day_before = sum_data(all_data, latest_date, -1, "confirmed")
# cases_difference = confirmed_cases_sum - cases_day_before
# cases_operand = "+" if cases_difference > 0 else "-"
# cases_percent_change = round((cases_difference / cases_day_before) * 100, 2)

# # number of deaths
# confirmed_deaths_sum = sum_data(all_data, latest_date, 0, "deaths")
# deaths_day_before = sum_data(all_data, latest_date, -1, "deaths")
# deaths_difference = confirmed_deaths_sum - deaths_day_before
# deaths_operand = "+" if deaths_difference > 0 else "-"
# deaths_percent_change = round((deaths_difference / deaths_day_before) * 100, 2)


# # number of recovered cases
# confirmed_recovered_sum = sum_data(all_data, latest_date, 0, "recovered")
# recovered_day_before = sum_data(all_data, latest_date, -1, "recovered")
# recovered_difference = confirmed_recovered_sum - recovered_day_before
# recovered_operand = "+" if recovered_difference > 0 else "-"
# recovered_percent_change = round((recovered_difference / recovered_day_before) * 100, 2)


# # number of active cases
# confirmed_active_cases = (
#     confirmed_cases_sum - confirmed_deaths_sum - confirmed_recovered_sum
# )

# # other key numbers
# outbreak_beginning = datetime.date(2019, 12, 31)
# days_since_outbreak = (datetime.date.today() - outbreak_beginning).days
# incident_fatality_rate = round((confirmed_deaths_sum / confirmed_cases_sum) * 100, 2)
# recovery_rate = round((confirmed_recovered_sum / confirmed_cases_sum) * 100, 2)

# # infected countries
# infected_countries = len(all_data["Country/Region"].unique())

In [4]:
print(all_data['date'].min())

2020-01-22


In [5]:
marks =[i for i in date_options]
print(marks)

['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020

In [6]:
options =[]
for date in list(date_options):
    options.append({"label":date,"value":date})

# print(options)


# print(all_data.dtypes)

In [7]:
daterange = pd.date_range(start=all_data['date'].min(),end=all_data['date'].max(),freq='d')

def unixTimeMillis(dt):
    ''' Convert datetime to unix timestamp '''
    return int(time.mktime(dt.timetuple()))

def unixToDatetime(unix):
    ''' Convert unix timestamp to datetime. '''
    return pd.to_datetime(unix,unit='s')

def getMarks(start, end):
    ''' Returns the marks for labeling.
    Every Nth value will be used.
    '''

    result = {}
    for i, date in enumerate(daterange):
#         print(date)
        # Append value to dict
        result[unixTimeMillis(date)] = str(date.strftime('%Y-%m-%d'))

    return result

In [8]:
value = getMarks(daterange.min(),daterange.max())
print(value)

{1579680000: '2020-01-22', 1579766400: '2020-01-23', 1579852800: '2020-01-24', 1579939200: '2020-01-25', 1580025600: '2020-01-26', 1580112000: '2020-01-27', 1580198400: '2020-01-28', 1580284800: '2020-01-29', 1580371200: '2020-01-30', 1580457600: '2020-01-31', 1580544000: '2020-02-01', 1580630400: '2020-02-02', 1580716800: '2020-02-03', 1580803200: '2020-02-04', 1580889600: '2020-02-05', 1580976000: '2020-02-06', 1581062400: '2020-02-07', 1581148800: '2020-02-08', 1581235200: '2020-02-09', 1581321600: '2020-02-10', 1581408000: '2020-02-11', 1581494400: '2020-02-12', 1581580800: '2020-02-13', 1581667200: '2020-02-14', 1581753600: '2020-02-15', 1581840000: '2020-02-16', 1581926400: '2020-02-17', 1582012800: '2020-02-18', 1582099200: '2020-02-19', 1582185600: '2020-02-20', 1582272000: '2020-02-21', 1582358400: '2020-02-22', 1582444800: '2020-02-23', 1582531200: '2020-02-24', 1582617600: '2020-02-25', 1582704000: '2020-02-26', 1582790400: '2020-02-27', 1582876800: '2020-02-28', 1582963200:

In [9]:
# data = [trace_line,trace_line2]
# # print(data)

# layout = dict(title="COVID",
#               showlegend=False)
# fig = dict(data = data, layout = layout)
from dateutil.relativedelta import relativedelta



app = dash.Dash()
app.layout = html.Div([
    html.Div(html.H1(children="COVID-19 Analysis")),

    
    html.Div(
    html.Label([dcc.RangeSlider(id = "date-slider", min = unixTimeMillis(daterange.min()),max = unixTimeMillis(daterange.max()),value = [unixTimeMillis(daterange.min()),unixTimeMillis(daterange.max())],marks=getMarks(daterange.min(),daterange.max())
                )])
              ),
#     html.Label([dcc.RangeSlider(
#             id = 'date-slider',
#             updatemode = 'mouseup', #don't let it update till mouse released
#             min = all_data['date'].min(),
#             max = all_data['date'].max(),
#             value = [all_data['date'].max()],
#             #TODO add markers for key dates
#             marks=get_marks_from_start_end(all_data['date'].min(),
#                                            all_data['date'].max()),
#         )]
#         ),
    
    html.Div(
        dcc.Graph(id="date_covid"
             ),
            style = {'overflowX':'scroll'}
        )

],style= {
                      "color": "black",
                      "text-align": "center","background-color": "yellow",
                      "border-style": "Bold","display":"inline-block","width":"100%",
                      })

@app.callback(
    dash.dependencies.Output("date_covid","figure"),
    [dash.dependencies.Input("date-slider","value")],
)
def update_fig(input_value):
    return 'From {} to {}'.format(unixToDatetime(input_value[0]),unixToDatetime(input_value[1]))
#     date_all_data = all_data[all_data['date']==input_value]
#     #print(date_all_data)
#     date_all_data=date_all_data.drop('Province/State', axis=1).groupby("Country/Region").sum().reset_index()
# #     data_fig_deaths=[]
#     trace_line1 = go.Bar(x=countries,
#                             y=list(date_all_data["deaths"]),
#                             name="Death",
#                             showlegend=False)
#     trace_line2 = go.Bar(x=countries,
#                             y=list(date_all_data["confirmed"]),
#                             name="Confirmed",
#                             showlegend=False)
#     trace_line3 = go.Bar(x=countries,
#                             y=list(date_all_data["recovered"]),
#                             name="Recovered",
#                             showlegend=False)
    
    
#     return {
#         "data":[trace_line1,trace_line2,trace_line3],
#         "layout":
#         go.Layout(
#             title='Datewise count',
#             barmode = 'stack')
#     }
    

if __name__=="__main__":
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2020 15:50:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2020 15:50:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2020 15:50:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2020 15:50:38] "POST /_dash-update-component HTTP/1.1" 200 -
